In [1]:
from src.data.dataset_functions import get_dataframei_onlymods

mod_dataframe = get_dataframei_onlymods(4) #Der Parameter steht für den Datensatz, der verwendet werden soll

providing ../data/raw/2022_place_canvas_history-000000000004.csv ...
../data/raw/2022_place_canvas_history-000000000004.csv is already in data/raw
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-13e9cb61-1c51-4ce3-972a-c5ac1f7f425f;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 69ms :: artifacts dl 3ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

<div style="color:#246fd6">
Im folgenden sollen die Statistiken für Mods bestimmt werden:


1) Wie oft mussten Mods zensieren im jeweiligen Datensatz?

2) Wie viele Pixel wurden insgesamt von einem Mod im jeweiligen Datensatz zensiert?

3) Werden Regionen deutlich, in denen Mods häufiger eingreifen mussten?

    3.1) Wenn ja, welche Regionen und welchen Artworks ist das zugeordnet?

    3.2) Zu bestimmen, welche Pixel mehr als einmal zensiert wurden, also min. 2-mal.

    -> wird von diesem Ansatz her verworfen!!!

    -> Es soll nun ein Canvas erstellt werden, in dem die Quadrate der Mods nachgezeichnet werden sollen in verschiedenen Farben,
    um so evtl. Gebiete direkt optisch darstellen zu können.

4) Evtl.: Wurden von Bots gesetzte Pixel zensiert?

-> viel zu umfangreich & komplex, genauso wie das Nachzeichnen der von Mods zensierten Artworks,
da dies viele Apsekte verücksichtigen muss, wie welche Pixel zuletzt vorher gesetzt wurden etc.

</div>

<div style="color:#246fd6">
Zur Beantwortung der ersten Frage:

Mods sind im "StandardDataFrame" nicht enthalen, nur noch im spezifischen ModsDataFrame!

Um die Frage zu beantworten muss gezählt werden, wie viele Einträge für Mods im jeweiligen Datensatz exitieren.
D.h. auch Duplikate von User_IDs sind relevant, sodass einfach nur gezählt werden muss, wie viele Einträge insgesamt im ModsDataFrame existieren.

Das wäre dann auch schon die Antwort darauf, wie oft Mods im jeweiligen Datensatz zensieren mussten.
</div>

In [2]:
#Allgemein die Häufigkeit, wie oft in einem Datensatz zensiert wurde
amount_of_censorship = mod_dataframe.count()
print(amount_of_censorship)

3


In [3]:
#Anzahl der Mods, die im Datensatz aktiv waren (Duplikate wurden entfernt)
amount_of_mods = mod_dataframe.drop('x1').drop('y1').drop('x2').drop('y2').drop('t').drop('pixel_color')
amount_of_mods = amount_of_mods.distinct().count()
print(amount_of_mods)

3


<div style="color:#246fd6">
Beantwortung der 2. Frage:
Betrachtung der zensierten Pixel pro Mod.

D.h. hier muss jeder Mod einzeln betrachtet werden und dabei noch berechnet werden, wie viele Pixel dabei das Quadrat aufgespannt haben.

Berechnung der Pixel: 
x1,y1 => obere linke Ecke des Quadrates 
x2,y2 => untere rechte Ecke des Quadrates

Damit ergibt sich:
```
 x1,y1----------------------x2,y1
      |                    |
      |                    |
      |                    |
    a |                    |
      |                    |
      |                    |
      |                    |
 x1,y2----------------------x2,y2
                b
```
Dann ergibt a * b die Anzahl an Pixel, die dabei berührt wurden und somit auch die Anzahl der Pixel, die dabei zensiert wurden.

a = y2 - y1

b = x2 - x1

Anzahl der zensierten Pixel pro Eintrag = (y2 - y1) * (x2 - x1)
</div>

In [4]:
#Vorbereitung um Anzahl der zensierten Pixel pro Mod zu bestimmen
pixel_amount = mod_dataframe.withColumn("amount",(mod_dataframe.y2 - mod_dataframe.y1) * (mod_dataframe.x2 - mod_dataframe.x1))
#pixel_amount = pixel_amount.orderBy('x','y')
pixel_amount.printSchema()
pixel_amount.show()

root
 |-- user_id: string (nullable = true)
 |-- x1: integer (nullable = true)
 |-- y1: integer (nullable = true)
 |-- x2: integer (nullable = true)
 |-- y2: integer (nullable = true)
 |-- t: long (nullable = true)
 |-- pixel_color: string (nullable = true)
 |-- amount: integer (nullable = true)



+--------------------+---+----+---+----+----------+-----------+------+
|             user_id| x1|  y1| x2|  y2|         t|pixel_color|amount|
+--------------------+---+----+---+----+----------+-----------+------+
|gXOuxCyb/dslyRb5G...| 23|1523|172|1792|1649045540|    #000000| 40081|
|Tqn87vyPxAHtPMu4x...| 51|1691|154|1807|1649046581|    #000000| 11948|
|oPv1iZrj4UMjfhzOT...| 44|1652|165|1899|1649045770|    #000000| 29887|
+--------------------+---+----+---+----+----------+-----------+------+



In [5]:
#Anzahl der zensierten Pixel pro Mod zusammengefasst würden jeden Mod
amount_per_mod = pixel_amount.groupBy('user_id').sum('amount')
amount_per_mod.show()

+--------------------+-----------+
|             user_id|sum(amount)|
+--------------------+-----------+
|gXOuxCyb/dslyRb5G...|      40081|
|oPv1iZrj4UMjfhzOT...|      29887|
|Tqn87vyPxAHtPMu4x...|      11948|
+--------------------+-----------+



<div style="color:#246fd6">
Nochmal über alle Mods summiert:
</div>

In [6]:
#Anzahl der insgesamt zensierten Pixel im jeweiligen Datensatz über alle Mods summiert
amounts = pixel_amount.agg({'amount': 'sum'}).show()

+-----------+
|sum(amount)|
+-----------+
|      81916|
+-----------+



In [7]:
#Im Verhältnis dazu bestimmen, wie viele Pixel von "normalen Nutzern" dagegen platziert wurden im Datensatz?
from src.data.dataset_functions import get_dataframe

dataFrame = get_dataframe(True)
dataFrame.printSchema()

providing ../data/raw/2022_place_canvas_history-000000000004.csv ...
../data/raw/2022_place_canvas_history-000000000004.csv is already in data/raw
root
 |-- user_id: string (nullable = true)
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- t: long (nullable = true)
 |-- pixel_color: string (nullable = true)



In [8]:
#Um herauszufinden, wie viele Pixel insgesamt platziert wurden, müssen nur alle Zeilen gezählt werden, 
# da eine Zeile = 1 platzierter Pixel
amount_placed_pixel = dataFrame.count()
print(amount_placed_pixel)

32772070


<div style="color:#246fd6">
Beantwortung der 3.Frage:
Regionen bestimmen, in denen häufiger eingegriffen wurde.

Dafür Überschneidungen zwischen den Pixeln der Einträge bestimmen.

Zu beachten dabei: Es müssen alle von der Zensur berührten Pixel betrachtet werden bei der Überschneidung.
Es müssen verschiedene Fälle betrachtet werden:

1) Eins der beiden Quadrate liegt genau im anderen.

    Dann ist x1b < x1s & x2s < x2b & y1b < y1s & y2b > y2s

    s => Quadrat s das vollständig im Quadrat b liegt

2) Teilweise Überschneidungen.

    Hier wirds schwierig, denn hier dürfen nur die in der Überschneidung liegenden Gebiete als "Mehrfach zensiert" gelabelt werden.
</div>

<div style="color:#17dd84">
Da das mit den Fallunterscheidungen zu komplex wird, mache ich ein Canvas (wie Greta bereits) und zeichne dort die Quadrate ein,

dann sieht man wo sich Quadrate überschneiden anhand unterschiedlicher Farben,

denn Quadrate werden nur mit sehr geringer Wahrscheinlichkeit gleich groß sein mit genau den selben Koordinaten.

Es sollte daher was zu sehen sein.
</div>

Inner-Join, sodass paarweiser Verlgeich zwischen Mods möglich ist

In [4]:
import pyspark.sql.functions as F

if(amount_of_censorship > 1):
    selfjoinedFrame = mod_dataframe.alias('df1').join(mod_dataframe.alias('df2'))
    selfjoinedFrame = selfjoinedFrame.where('df2.user_id <> df1.user_id')
# selfjoinedFrame = selfjoinedFrame.where('Abs(df2.x - df1.x) <= 2*'+str(RZModel.max_xy_dist))
# selfjoinedFrame = selfjoinedFrame.where('Abs(df2.y - df1.y) <= 2*'+str(RZModel.max_xy_dist))
# selfjoinedFrame = selfjoinedFrame.select(F.col('df1.user_id'),F.col('df1.x').alias("x1"),F.col('df2.x').alias('x2'),F.col('df1.y').alias("y1"),F.col('df2.y').alias('y2'),F.col('df1.t').alias("t1"),F.col('df2.t').alias('t2'),F.col('df1.pixel_color').alias("c1"),F.col('df2.pixel_color').alias('c2'))
selfjoinedFrame.show()

+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+
|             user_id| x1|  y1| x2|  y2|         t|pixel_color|             user_id| x1|  y1| x2|  y2|         t|pixel_color|
+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+
|gXOuxCyb/dslyRb5G...| 23|1523|172|1792|1649045540|    #000000|Tqn87vyPxAHtPMu4x...| 51|1691|154|1807|1649046581|    #000000|
|gXOuxCyb/dslyRb5G...| 23|1523|172|1792|1649045540|    #000000|oPv1iZrj4UMjfhzOT...| 44|1652|165|1899|1649045770|    #000000|
|Tqn87vyPxAHtPMu4x...| 51|1691|154|1807|1649046581|    #000000|gXOuxCyb/dslyRb5G...| 23|1523|172|1792|1649045540|    #000000|
|Tqn87vyPxAHtPMu4x...| 51|1691|154|1807|1649046581|    #000000|oPv1iZrj4UMjfhzOT...| 44|1652|165|1899|1649045770|    #000000|
|oPv1iZrj4UMjfhzOT...| 44|1652|165|1899|1649045770|    #000000|gXOuxCyb/dslyRb5G...| 23|1523|172|1792|1649045540|    #

Jetzt schauen, ob's überhaupt Überschneidungen gibt

In [6]:
pair_comp = selfjoinedFrame.where('(df1.x1 < df2.x1 AND df2.x1 < df1.x2 AND df1.y1 < df2.y1 AND df2.y1 < df1.y2) OR (df1.x1 < df2.x2 AND df2.x2 < df1.x2 AND df1.y1 < df2.y1 AND df2.y1 < df1.y2) OR (df1.x1 < df2.x1 AND df2.x1 < df1.x2 AND df1.y1 < df2.y2 AND df2.y2 < df1.y2) OR (df1.x1 < df2.x2 AND df2.x2 < df1.x2 AND df1.y1 < df2.y2 AND df2.y2 < df1.y2)')
pair_comp.show()

+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+
|             user_id| x1|  y1| x2|  y2|         t|pixel_color|             user_id| x1|  y1| x2|  y2|         t|pixel_color|
+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+
|gXOuxCyb/dslyRb5G...| 23|1523|172|1792|1649045540|    #000000|Tqn87vyPxAHtPMu4x...| 51|1691|154|1807|1649046581|    #000000|
|gXOuxCyb/dslyRb5G...| 23|1523|172|1792|1649045540|    #000000|oPv1iZrj4UMjfhzOT...| 44|1652|165|1899|1649045770|    #000000|
|oPv1iZrj4UMjfhzOT...| 44|1652|165|1899|1649045770|    #000000|Tqn87vyPxAHtPMu4x...| 51|1691|154|1807|1649046581|    #000000|
+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+



Jetzt müssen die Überschneidungen grob bestimmt werden, sodass Gebiete besser benannt werden können.

Die Gebiete sollte jedoch nicht mit 100% Genauigkeit benannt werden, da dies schwierig wird.

Anhand des DataFrames pair_comp weiß man nun, dass df2.user_id in df1.user_id liegen muss.

Dadurch vereinfacht sich der Vergleich, um die Gebiete genauer zu bestimmen.

In [8]:
#1. Fall, Quadrat von df2 liegt genau in df1
#Dann ist x1b < x1s & x2s < x2b & y1b < y1s & y2b > y2s
#Erhält man hier ein Ergebnis, so wurden die Pixel des Quadrates aus df2 mindestens 2 Mal zensiert!
#Hier kann also ein genaues Gebiet angegeben werden
case_one = pair_comp.where('df1.x1 < df2.x1 AND df2.x2 < df1.x2 AND df1.y1 < df2.y1 AND df2.y2 < df1.y2')
case_one.show()

+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+
|             user_id| x1|  y1| x2|  y2|         t|pixel_color|             user_id| x1|  y1| x2|  y2|         t|pixel_color|
+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+
|oPv1iZrj4UMjfhzOT...| 44|1652|165|1899|1649045770|    #000000|Tqn87vyPxAHtPMu4x...| 51|1691|154|1807|1649046581|    #000000|
+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+



Nun kommen weitere Fallunterscheidungen.

2.Fall: die linke obere Ecke von df2 ist in der rechten unteren Ecke von df1.

In [9]:
case_two = pair_comp.where('df2.x1 < df1.x2 AND df1.x2 < df2.x2 AND df2.y1 < df2.y2 AND df1.y2 < df2.y2')
case_two.show()

+-------+---+---+---+---+---+-----------+-------+---+---+---+---+---+-----------+
|user_id| x1| y1| x2| y2|  t|pixel_color|user_id| x1| y1| x2| y2|  t|pixel_color|
+-------+---+---+---+---+---+-----------+-------+---+---+---+---+---+-----------+
+-------+---+---+---+---+---+-----------+-------+---+---+---+---+---+-----------+



3.Fall: linke untere Ecke von df2 ist in df1

In [10]:
case_three = pair_comp.where('df1.x2 < df2.x2 AND df1.x1 < df2.x1 AND df2.y2 > df1.y1 AND df2.y2 < df1.y2')
case_three.show()

+-------+---+---+---+---+---+-----------+-------+---+---+---+---+---+-----------+
|user_id| x1| y1| x2| y2|  t|pixel_color|user_id| x1| y1| x2| y2|  t|pixel_color|
+-------+---+---+---+---+---+-----------+-------+---+---+---+---+---+-----------+
+-------+---+---+---+---+---+-----------+-------+---+---+---+---+---+-----------+



4.Fall: rechte obere Ecke von df2 ist in df1.

In [11]:
case_four = pair_comp.where('df1.x1 < df2.x2 AND df2.x2 < df1.x2 AND df1.y1 < df2.y1 AND df2.y1 < df1.y2')
case_four.show()

+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+
|             user_id| x1|  y1| x2|  y2|         t|pixel_color|             user_id| x1|  y1| x2|  y2|         t|pixel_color|
+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+
|gXOuxCyb/dslyRb5G...| 23|1523|172|1792|1649045540|    #000000|Tqn87vyPxAHtPMu4x...| 51|1691|154|1807|1649046581|    #000000|
|gXOuxCyb/dslyRb5G...| 23|1523|172|1792|1649045540|    #000000|oPv1iZrj4UMjfhzOT...| 44|1652|165|1899|1649045770|    #000000|
|oPv1iZrj4UMjfhzOT...| 44|1652|165|1899|1649045770|    #000000|Tqn87vyPxAHtPMu4x...| 51|1691|154|1807|1649046581|    #000000|
+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+



5.Fall: rechte untere Ecke von df2 ist in df1.

In [12]:
case_five = pair_comp.where('df1.x1 < df2.x2 AND df2.x2 < df1.x2 AND df1.y1 < df2.y2 AND df2.y2 < df1.y2')
case_five.show()

+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+
|             user_id| x1|  y1| x2|  y2|         t|pixel_color|             user_id| x1|  y1| x2|  y2|         t|pixel_color|
+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+
|oPv1iZrj4UMjfhzOT...| 44|1652|165|1899|1649045770|    #000000|Tqn87vyPxAHtPMu4x...| 51|1691|154|1807|1649046581|    #000000|
+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+



6.Fall: die komplette rechte Seite von df2 ist in df1.

In [13]:
case_six = pair_comp.where('df1.x1 < df2.x2 AND df2.x2 < df1.x2 AND df1.y1 < df2.y1 AND df2.y2 < df1.y2')
case_six.show()

+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+
|             user_id| x1|  y1| x2|  y2|         t|pixel_color|             user_id| x1|  y1| x2|  y2|         t|pixel_color|
+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+
|oPv1iZrj4UMjfhzOT...| 44|1652|165|1899|1649045770|    #000000|Tqn87vyPxAHtPMu4x...| 51|1691|154|1807|1649046581|    #000000|
+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+



7.Fall: die untere Seite von df2 ist in df1.

In [14]:
case_seven = pair_comp.where('df1.x1 < df2.x1 AND df2.x2 < df1.x2 AND df1.y1 < df2.y2 AND df2.y2 < df1.y2')
case_seven.show()

+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+
|             user_id| x1|  y1| x2|  y2|         t|pixel_color|             user_id| x1|  y1| x2|  y2|         t|pixel_color|
+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+
|oPv1iZrj4UMjfhzOT...| 44|1652|165|1899|1649045770|    #000000|Tqn87vyPxAHtPMu4x...| 51|1691|154|1807|1649046581|    #000000|
+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+



8.Fall: die linke Seite von df2 ist in df1.

In [15]:
case_eight = pair_comp.where('df1.x1 < df2.x1 AND df2.x1 < df1.x2 AND df1.y1 < df2.y1 AND df2.y2 < df1.y2')
case_eight.show()

+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+
|             user_id| x1|  y1| x2|  y2|         t|pixel_color|             user_id| x1|  y1| x2|  y2|         t|pixel_color|
+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+
|oPv1iZrj4UMjfhzOT...| 44|1652|165|1899|1649045770|    #000000|Tqn87vyPxAHtPMu4x...| 51|1691|154|1807|1649046581|    #000000|
+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+



Letzter Fall 9. : obere Seite von df2 ist in df1.

In [16]:
case_nine = pair_comp.where('df1.x1 < df2.x1 AND df2.x2 < df1.x2 AND df1.y1 < df2.y1 AND df2.y1 < df1.y2')
case_nine.show()

+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+
|             user_id| x1|  y1| x2|  y2|         t|pixel_color|             user_id| x1|  y1| x2|  y2|         t|pixel_color|
+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+
|gXOuxCyb/dslyRb5G...| 23|1523|172|1792|1649045540|    #000000|Tqn87vyPxAHtPMu4x...| 51|1691|154|1807|1649046581|    #000000|
|gXOuxCyb/dslyRb5G...| 23|1523|172|1792|1649045540|    #000000|oPv1iZrj4UMjfhzOT...| 44|1652|165|1899|1649045770|    #000000|
|oPv1iZrj4UMjfhzOT...| 44|1652|165|1899|1649045770|    #000000|Tqn87vyPxAHtPMu4x...| 51|1691|154|1807|1649046581|    #000000|
+--------------------+---+----+---+----+----------+-----------+--------------------+---+----+---+----+----------+-----------+

